In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [4]:
# load the trained model
model = load_model('model.h5')

# load the encoder
with open('label_encoder_gender.pkl', 'rb') as file:
    lable_encoder_gender = pickle.load(file)

with open('one_hot_encoder_geography.pkl','rb') as file:
    one_hot_encoder_geography = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [5]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [6]:
# One-hot encode 'Geography'
geo_encoded = one_hot_encoder_geography.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geography.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [8]:
## Encode categorical variables
input_df['Gender']=lable_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [9]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [10]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.52544045,  0.90750738,  0.10007155, -0.6962018 , -0.2629485 ,
         0.80883036,  0.64104192,  0.97067965, -0.86742511,  1.00175153,
        -0.57831252, -0.57773517]])

In [11]:
## PRedict churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 99ms/step


array([[0.04316921]], dtype=float32)

In [13]:
prediction_proba = prediction[0][0]
prediction_proba

0.043169208

In [14]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
